In [0]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
% pip install pomegranate
#Pomegranate is a graphical models library for Python, implemented in Cython for speed

     |████████████████████████████████| 3.3MB 2.8MB/s 
  Created wheel for pomegranate: filename=pomegranate-0.13.3-cp36-cp36m-linux_x86_64.whl size=10950002 sha256=d05b25ec82387428f0bdfd38cb40e7df8c32f37dbb81c2cac39a88ce686bc74c
  Stored in directory: /root/.cache/pip/wheels/92/a4/39/0f9e71a9134d03801477bffa6e02257020f12e5cbb031a8489
Successfully built pomegranate


In [0]:
import sys
sys.path.append("/content/drive/My Drive/Colab Notebooks/assignment")

In [0]:
from IPython.core.display import HTML
from itertools import chain
from collections import Counter,defaultdict

In [0]:
from helpers import show_model, Dataset

In [8]:
data = Dataset("/content/drive/My Drive/Colab Notebooks/assignment/tags-universal.txt", "/content/drive/My Drive/Colab Notebooks/assignment/brown-universal.txt", train_test_split=0.8)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

There are 57340 sentences in the corpus.
There are 45872 sentences in the training set.
There are 11468 sentences in the testing set.


In [9]:
key = 'b100-38532'
print("Sentence: {}".format(key))
print("words:\n\t{!s}".format(data.sentences[key].words))
print("tags:\n\t{!s}".format(data.sentences[key].tags))

Sentence: b100-38532
words:
	('Perhaps', 'it', 'was', 'right', ';', ';')
tags:
	('ADV', 'PRON', 'VERB', 'ADJ', '.', '.')


In [10]:
print("There are a total of {} samples of {} unique words in the corpus."
      .format(data.N, len(data.vocab)))
print("There are {} samples of {} unique words in the training set."
      .format(data.training_set.N, len(data.training_set.vocab)))
print("There are {} samples of {} unique words in the testing set."
      .format(data.testing_set.N, len(data.testing_set.vocab)))
print("There are {} words in the test set that are missing in the training set."
      .format(len(data.testing_set.vocab - data.training_set.vocab)))

assert data.N == data.training_set.N + data.testing_set.N, \
       "The number of training + test samples should sum to the total number of samples"

There are a total of 1161192 samples of 56057 unique words in the corpus.
There are 928458 samples of 50536 unique words in the training set.
There are 232734 samples of 25112 unique words in the testing set.
There are 5521 words in the test set that are missing in the training set.


In [11]:
# accessing words with Dataset.X and tags with Dataset.Y 
for i in range(5):    
    print("Sentence {}:".format(i + 1), data.X[i])
    print()
    print("Labels {}:".format(i + 1), data.Y[i])
    print()

Sentence 1: ('Mr.', 'Podger', 'had', 'thanked', 'him', 'gravely', ',', 'and', 'now', 'he', 'made', 'use', 'of', 'the', 'advice', '.')

Labels 1: ('NOUN', 'NOUN', 'VERB', 'VERB', 'PRON', 'ADV', '.', 'CONJ', 'ADV', 'PRON', 'VERB', 'NOUN', 'ADP', 'DET', 'NOUN', '.')

Sentence 2: ('But', 'there', 'seemed', 'to', 'be', 'some', 'difference', 'of', 'opinion', 'as', 'to', 'how', 'far', 'the', 'board', 'should', 'go', ',', 'and', 'whose', 'advice', 'it', 'should', 'follow', '.')

Labels 2: ('CONJ', 'PRT', 'VERB', 'PRT', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADP', 'ADV', 'ADV', 'DET', 'NOUN', 'VERB', 'VERB', '.', 'CONJ', 'DET', 'NOUN', 'PRON', 'VERB', 'VERB', '.')

Sentence 3: ('Such', 'an', 'instrument', 'is', 'expected', 'to', 'be', 'especially', 'useful', 'if', 'it', 'could', 'be', 'used', 'to', 'measure', 'the', 'elasticity', 'of', 'heavy', 'pastes', 'such', 'as', 'printing', 'inks', ',', 'paints', ',', 'adhesives', ',', 'molten', 'plastics', ',', 'and', 'bread', 'dough', ',', 'for',

In [12]:
# use Dataset.stream() (word, tag) samples for the entire corpus
print("\nStream (word, tag) pairs:\n")
for i, pair in enumerate(data.stream()):
    print("\t", pair)
    if i > 10: break


Stream (word, tag) pairs:

	 ('Mr.', 'NOUN')
	 ('Podger', 'NOUN')
	 ('had', 'VERB')
	 ('thanked', 'VERB')
	 ('him', 'PRON')
	 ('gravely', 'ADV')
	 (',', '.')
	 ('and', 'CONJ')
	 ('now', 'ADV')
	 ('he', 'PRON')
	 ('made', 'VERB')
	 ('use', 'NOUN')


In [13]:
from collections import Counter,defaultdict

def pair_counts(tags,words):
	d=defaultdict(lambda: defaultdict(int))
	for tag,word in zip(tags,words):
		d[tag][word]+=1
	return d
	raise NotImplementedError
tags = [tag for i,(word, tag) in enumerate(data.training_set.stream())]
words = [word for i,(word, tag) in enumerate(data.training_set.stream())]

emission_counts =pair_counts(tags,words)

tags

['ADV',
 'NOUN',
 '.',
 'ADV',
 '.',
 'VERB',
 'ADP',
 'ADJ',
 'NOUN',
 'CONJ',
 'VERB',
 'ADJ',
 'NOUN',
 '.',
 'DET',
 'VERB',
 'ADJ',
 'PRT',
 'ADP',
 'NUM',
 'NOUN',
 '.',
 'PRON',
 'VERB',
 'PRT',
 'VERB',
 'NOUN',
 '.',
 'VERB',
 'NOUN',
 'NUM',
 '.',
 'NUM',
 '.',
 '.',
 '.',
 'ADP',
 'ADV',
 'NUM',
 'NOUN',
 '.',
 'DET',
 'NOUN',
 'CONJ',
 'DET',
 'NOUN',
 'VERB',
 'ADP',
 'NOUN',
 'VERB',
 'ADP',
 'DET',
 'NOUN',
 'ADP',
 'NOUN',
 'CONJ',
 'NOUN',
 '.',
 'NOUN',
 '.',
 'NOUN',
 '.',
 'NOUN',
 '.',
 'VERB',
 '.',
 'VERB',
 'ADP',
 'NOUN',
 '.',
 'VERB',
 '.',
 'VERB',
 '.',
 'VERB',
 '.',
 'VERB',
 '.',
 'ADJ',
 'NOUN',
 'ADV',
 '.',
 'NOUN',
 '.',
 'PRON',
 'VERB',
 'VERB',
 'ADP',
 'DET',
 'NOUN',
 '.',
 'ADP',
 'DET',
 'NOUN',
 '.',
 'ADV',
 'DET',
 'ADJ',
 'NOUN',
 'DET',
 'NOUN',
 'CONJ',
 'NOUN',
 'ADP',
 'NOUN',
 'VERB',
 'ADJ',
 '.',
 'NOUN',
 'VERB',
 'DET',
 'ADJ',
 'VERB',
 'NOUN',
 'NOUN',
 'ADP',
 'NOUN',
 'NUM',
 '.',
 'NUM',
 '.',
 'PRT',
 'DET',
 'NOUN',
 'DET',

In [0]:
# Create a lookup table mfc_table where mfc_table[word] contains the tag label most frequently assigned to that word
from collections import namedtuple

FakeState = namedtuple("FakeState", "name")

class MFCTagger:
    # NOTE: You should not need to modify this class or any of its methods
    missing = FakeState(name="<MISSING>")
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name=tag) for word, tag in table.items()})
        
    def viterbi(self, seq):
        """This method simplifies predictions by matching the Pomegranate viterbi() interface"""
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))


# TODO: calculate the frequency of each tag being assigned to each word (hint: similar, but not
# the same as the emission probabilities) and use it to fill the mfc_table

tags = [tag for i, (word, tag) in enumerate(data.training_set.stream())]
words = [word for i, (word, tag) in enumerate(data.training_set.stream())]
#Since this is the word_counts we will pass first words and then counts
word_counts = pair_counts(words,tags)

mfc_table = dict((word, max(tags.keys(), key=lambda key: tags[key])) for word, tags in word_counts.items())

# DO NOT MODIFY BELOW THIS LINE
mfc_model = MFCTagger(mfc_table) # Create a Most Frequent Class tagger instance

In [0]:
def replace_unknown(sequence):
    """Return a copy of the input sequence where each unknown word is replaced
    by the literal string value 'nan'. Pomegranate will ignore these values
    during computation.
    """
    return [w if w in data.training_set.vocab else 'nan' for w in sequence]

def simplify_decoding(X, model):
    """X should be a 1-D sequence of observations for the model to predict"""
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]  # do not show the start/end state predictions

In [16]:
for key in data.testing_set.keys[:5]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.')


Sentence Key: b100-35462

Predicted labels:
-----------------
['DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'ADJ', 'NOUN', '.', 'CONJ', 'ADP', 'DET', '<MISSING>', 'ADP', 'ADJ', 'ADJ', 

In [0]:
def accuracy(X, Y, model):
 
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags = simplify_decoding(observations, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

In [18]:
mfc_training_acc = accuracy(data.training_set.X, data.training_set.Y, mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

mfc_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))


training accuracy mfc_model: 95.72%
testing accuracy mfc_model: 93.01%


IMPLEMENTATION of Unigram


In [0]:
def unigram_counts(sequences):
    return Counter(sequences)
    raise NotImplementedError
tag_unigrams = unigram_counts(tags)

IMPLEMENTATION of Bigram

In [0]:
def bigram_counts(sequences):
    return Counter(zip(sequences, sequences[1:]))
    raise NotImplementedError

tag_bigrams = bigram_counts(tags)

IMPLEMENTATION of Sequence Starting Counts

In [0]:
def starting_counts(sequences):
    return Counter([x[0] for x in sequences])
    raise NotImplementedError
tag_starts = starting_counts(data.training_set.Y)

IMPLEMENTATION of Sequence Ending Counts

In [0]:
def ending_counts(sequences):
    return Counter([x[-1] for x in sequences])
    raise NotImplementedError

tag_ends = ending_counts(data.training_set.Y)

IMPLEMENTATION of HMM Tagger

In [0]:
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

In [0]:
basic_model = HiddenMarkovModel(name="base-hmm-tagger")

In [0]:
tag_counts = pair_counts(tags, words)
tag_state = {}
for tag in data.training_set.tagset:
    for word in data.training_set.vocab:
        try:
            tag_counts[tag][word] /= tag_unigrams[tag]
        except:
            tag_counts[tag][word] = 0
    emission = DiscreteDistribution(dict(tag_counts[tag]))
    tag_state[tag] = State(emission, name=tag)
basic_model.add_states(list(tag_state.values()))



In [0]:
tag_transition = defaultdict(lambda:0)
for tag in tag_unigrams.keys():
    for tag2 in tag_unigrams.keys():
        basic_model.add_transition(tag_state[tag], tag_state[tag2],
                                   tag_bigrams[(tag,tag2)] / tag_unigrams[tag])
    basic_model.add_transition(basic_model.start, tag_state[tag],
                              tag_starts[tag] / len(data.training_set))
    basic_model.add_transition(tag_state[tag], basic_model.end,
                              tag_ends[tag] / tag_unigrams[tag])

In [0]:
basic_model.bake()

In [42]:
observations = ['what', 'is', 'it']
forward_matrix = np.exp(basic_model.forward(observations))
probability_percentage = np.exp(basic_model.log_probability(observations))
print("        " + "".join(s.name.center(len(s.name)+6) for s in basic_model.states))
for i in range(len(observations) + 1):
    print(" <start> " if i==0 else observations[i - 1].center(9), end="")
    print("".join("{:.4f}%".format(100 * forward_matrix[i, j]).center(len(s.name) + 6)
                  for j, s in enumerate(basic_model.states)))

print("\nThe likelihood over all possible paths " + \
      "of this model producing the sequence {} is {:.10f}%\n\n"
      .format(observations, 100 * probability_percentage))

           .      ADJ      ADP      ADV      CONJ      DET      NOUN      NUM      PRON      PRT      VERB      X      base-hmm-tagger-start      base-hmm-tagger-end   
 <start> 0.0000% 0.0000%  0.0000%  0.0000%  0.0000%   0.0000%  0.0000%   0.0000%  0.0000%   0.0000%  0.0000%  0.0000%         100.0000%                  0.0000%         
   what  0.0000% 0.0000%  0.0000%  0.0000%  0.0000%   0.2197%  0.0000%   0.0000%  0.0000%   0.0000%  0.0000%  0.0000%          0.0000%                   0.0000%         
    is   0.0000% 0.0000%  0.0000%  0.0000%  0.0000%   0.0000%  0.0000%   0.0000%  0.0000%   0.0000%  0.0008%  0.0000%          0.0000%                   0.0000%         
    it   0.0000% 0.0000%  0.0000%  0.0000%  0.0000%   0.0000%  0.0000%   0.0000%  0.0000%   0.0000%  0.0000%  0.0000%          0.0000%                   0.0000%         

The likelihood over all possible paths of this model producing the sequence ['what', 'is', 'it'] is 0.0000000006%




In [44]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, basic_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.')


Sentence Key: b100-35462

Predicted labels:
-----------------
['DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'ADJ', 'NOUN', '.', 'CONJ', 'ADP', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', '.', 

In [43]:
hmm_training_acc = accuracy(data.training_set.X, data.training_set.Y, basic_model)
print("training accuracy basic hmm model: {:.2f}%".format(100 * hmm_training_acc))

hmm_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, basic_model)
print("testing accuracy basic hmm model: {:.2f}%".format(100 * hmm_testing_acc))

training accuracy basic hmm model: 97.52%
testing accuracy basic hmm model: 95.94%
